In [ ]:
!pip install ../input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl

In [ ]:
import pandas as pd
import keras
import os
import numpy as np
from sklearn.metrics import log_loss
from keras import Sequential
from keras.layers import *
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import cv2
from mtcnn import MTCNN
from tqdm.notebook import tqdm

In [ ]:
df_trains = pd.read_json('../input/deepfakes/metadata.json')
nums = list(range(len(df_trains)))

In [ ]:
from tqdm import tqdm_notebook
def read_image(num,name):
    num=str(num)
    if len(num)==2:
        path='../input/deepfakes/DeepFake'+num+'/DeepFake'+num+'/' + x.replace('.mp4', '') + '.jpg'
        return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    else:
        path='../input/deepfakes/DeepFake0'+num+'/DeepFake0'+num+'/' + x.replace('.mp4', '') + '.jpg'
        return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        
X = []
y = []
for df_train,num in tqdm_notebook(zip(df_trains,nums),total=len(df_trains)):
    images = list(df_train.columns.values)
    for x in images:
        try:
            X.append(read_image(num,x))
            y.append(LABELS.index(df_train[x]['label']))
        except Exception as err:
            print(x)

In [ ]:
def define_model():
    model = Sequential(
        [
            Conv2D(8, (3, 3), padding="same", activation = 'elu', input_shape=(92, 92,3)),
            BatchNormalization(),
            MaxPooling2D(2, 2),
            Conv2D(8, (5, 5), padding="same", activation = 'elu'),
            BatchNormalization(),
            MaxPooling2D(2, 2),
            Conv2D(16, (5, 5), padding="same", activation = 'elu'),
            BatchNormalization(),
            MaxPooling2D(2, 2),
            Conv2D(16, (5, 5), padding="same", activation = 'elu'),
            BatchNormalization(),
            MaxPooling2D(2, 2),
            Flatten(),
            Dropout(0.5),
            Dense(16,activation='relu'),
            Dropout(0.5),
            Dense(1, activation="sigmoid"),
        ]
    )
    model.compile(loss='mean_squared_error',optimizer=Adam(lr=5e-5))
    model.summary()
    return model

In [ ]:
model=define_model()
model.fit([train_X],[train_y],epochs=15)
model.save('model.h5')

In [ ]:
test_dir = '/kaggle/input/deepfake-detection-challenge/test_videos/'
filenames=os.listdir(test_dir)
test_video_files = [test_dir + x for x in filenames]
detector = MTCNN()
def detect_face(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final = []
    detected_faces_raw = detector.detect_faces(img)
    if detected_faces_raw == []:
        print('no faces found, skip to next frame')
        return []
    for x in detected_faces_raw:
        x, y, w, h = x['box']
        final.append([x, y, w, h])
    return final
def crop(img, x, y, w, h):
    x -= 40
    y -= 40
    w += 40
    h += 40
    if x < 0:
        x = 0
    if y <= 0:
        y = 0
    return cv2.cvtColor(cv2.resize(img[y: y + h, x: x + w], (92, 92)), cv2.COLOR_BGR2RGB)
def detect_video(video):
    cap = cv2.VideoCapture(video)
    ret, frame = cap.read()
    while True:
        ret, frame = cap.read()
        bounding_box = detect_face(frame)
        if bounding_box == []:
            continue
        x, y, w, h = bounding_box[0]
        return crop(frame, x, y, w, h)
test_X = []
for video in tqdm(test_video_files):
    test_X.append(detect_video(video))

In [ ]:
df_test=pd.read_csv('/kaggle/input/deepfake-detection-challenge/MesoNet.csv')
pred=model.predict([test_X])
df_test['label']=pred
df_test['filename']=filenames
df_test.to_csv('MesoNet.csv',index=False)